# Search Algortihms

To best understand an algorithm, build it. 

Search is an integral part of AI. It helps in problem solving across a wide variety of domains where a solution isn’t immediately clear. The Breadth First Search algortihm, A\* and Djikstra's are popular search algortihms, however their implementation is non-trivial. This certainly holds tru for optimal bi-directional and tri-directional search problems. With that in mind and to really appreciate the usage of libraries for search algorithms I will implement several graph search algorithms from scratch with the goal of ultimately designing an algorithm that can solve tri-directional search problems while achieving minimal node exploration count.

---

### Disclaimer and Copyright Notice

All functional implementations presented in this report have been independently developed by me, Franz Adam. If code or methodology, especially around my tri-directional UCS search algorithm, is used please reference me accoridngly. Since some of the code and solution presented in this report might overlap with graduate CS univeristy courses, I advise current students to adhere to their academic institution's policies regarding integrity and plagiarism before continuing. 

## The Search Graph 

As inspiration from Peter Norvigs famous textbook, Artificial Intelligence, I ran, traced and tested my search algorithms to calculate a route between two points in Romania while seeking to minimize time and space cost. 

Additionally, since the Romania graph is relatively small, I utilized an additional graph of Atlanta with many more nodes for further testing. 
![romania.png](images/romania.png)

In [2]:
import heapq
import os
import pickle
import math

### Priority Queue

In all searches that involve calculating path cost or heuristic (e.g. uniform-cost), we have to order our search frontier. It turns out the way that we do this can impact our overall search runtime. To show this, I will implement a priority queue which will help in understanding its performance benefits. For large graphs, sorting all input to a priority queue is impractical. As such, the data structure I implement will have an **amortized O(1) insertion and O(lg n) removal time**. For the purposes of this project, I treat **smaller values as values with higher priority**. For example a value of 1 has a higher priority than a value of 2.


#### Mathematical Overview:

A priority queue, denoted as $Q$, operates by inserting nodes $n$ with an associated priority. The operation can be defined as:

$$Q.insert(n, priority)$$

In the context of search algorithms, the priority typically depends on the cost $g(n)$ or the evaluation function $f(n) = g(n) + h(n)$, where $h(n)$ is the heuristic.

#### Implementation Details:

The priority queue was implemented using a binary heap to efficiently support the operations required for the search algorithms, particularly the extract-min operation, which is critical for performance.

**Properties**
1. Amortized O(1) insertion and O(log n) removal. 
2. Preserve FIFO. It is possible for the priority queue to receive two elements with the same priority. In the event that that occurs, the element that joined the priority queue first should be returned first.
3. Generalize. The nodes provided to the priority queue will be Python tuples in the form of `(priority, payload)`. We assume that the datatype for `priority` is a standard Python datatype (i.e. `int`, `float`, `str`, etc.). The `payload` can be of any datatype (standard or custom).
4. It is possible for duplicate nodes to enter the queue. (i.e. identical priority, identical payload)

In [73]:
class PriorityQueue(object):
    """
    A queue structure where each element is served in order of priority.

    Elements in the queue are popped based on the priority with higher priority
    elements being served before lower priority elements.  If two elements have
    the same priority, they will be served in the order they were added to the
    queue.

    Traditionally priority queues are implemented with heaps, but there are any
    number of implementation options.

    (Hint: take a look at the module heapq)

    Attributes:
        queue (list): Nodes added to the priority queue.
    """

    def __init__(self):
        """Initialize a new Priority Queue."""

        self.queue = []
        self.counter = 0

    def pop(self):
       
        """
        Pop the top priority node from the queue.

        Returns:
        The node with the highest priority.
        """
        if self.size() == 0: #Check for empty
            raise IndexError("There are no items in the queue.")
        
        elif self.size() == 1: #I hardcoded this base case to simplify the implementation
            return_node = self.queue[0]
            self.queue.pop(0)
        
        else: 
            return_node = self.queue[0]
            self.queue[0], heapify_index = self.queue[-1], 0 #Swap last and first item and then heapify it downwards
            self.queue.pop(-1) #Delete/Pop the last item 
        
            index_left_child = heapify_index * 2 + 1 #Set up child indixes for heapified node
            index_right_child = heapify_index * 2 + 2
        
            #While left or right children exist to potentially swap
            while index_left_child < self.size() or index_right_child < self.size(): 
                smallest = heapify_index

                #If left child exists and < than heapified node or if == same and counter is smaller, set smallest to left child
                if index_left_child < self.size() and ((self.queue[index_left_child][0] < self.queue[smallest][0]) 
                                                       or ((self.queue[index_left_child][0] == self.queue[smallest][0]) 
                                                           and (self.queue[index_left_child][1] < self.queue[smallest][1]))):
                    smallest = index_left_child
                #Same for the right
                if index_right_child < self.size() and ((self.queue[index_right_child][0] < self.queue[smallest][0]) 
                                                       or ((self.queue[index_right_child][0] == self.queue[smallest][0]) 
                                                           and (self.queue[index_right_child][1] < self.queue[smallest][1]))):
                    smallest = index_right_child

                if smallest == heapify_index: #Break if heapified node 
                    break

                # Swap with the smallest child
                self.queue[heapify_index], self.queue[smallest] = self.queue[smallest], self.queue[heapify_index]
                heapify_index = smallest

                index_left_child = heapify_index * 2 + 1
                index_right_child = heapify_index * 2 + 2

        return (return_node[0], return_node[2])
        
    def update(self, node, updated_priority):
        """
        Updates priority value of queue for A* and UCS to a smaller value if found in A*
        Loops through and updates. 
        """
        loop = True
        
        for i in range(len(self.queue)):
            
            if (self.queue[i][2] == node) and (loop == True):
                #current = node
                self.queue[i][0] = updated_priority
                
                if i != 0:
                    current_index = i
                    parent_index = int((current_index - 1) // 2)
                
                    #Heapfiy up
                    while (((self.queue[current_index][0] < 
                           self.queue[parent_index][0]) or (self.queue[current_index][0] == 
                           self.queue[parent_index][0] and self.queue[current_index][1] < 
                           self.queue[parent_index][1])) and (current_index != 0)):
                    
                        temp = self.queue[current_index]
                        self.queue[current_index] = self.queue[parent_index]
                        self.queue[parent_index] = temp
                           
                        #update current and parent index
                        current_index = parent_index
                        if current_index != 0:
                           parent_index = int((parent_index - 1) // 2)
                           
                    loop = False

    def __iter__(self):
        """Queue iterator."""

        return iter(sorted(self.queue))

    def append(self, node):
        """
        Append a node to the queue.

        Args:
            node: Comparable Object to be added to the priority queue.
        
        I am building a min-heap as implementation. I conserve the min-heap property by heapifying every new node 
        as long as it's smaller than it's parent.
        """
        #Restructure the tuple into a list to keep track of insertion order
        node_list = []
        node_list.append(node[0])
        node_list.append(self.counter)
        node_list.append(node[1])
        self.counter = self.counter + 1
        
        if self.size() == 0:
            self.queue.append(node_list)
        else: 
            self.queue.append(node_list)
            new_node_index = self.size() - 1
            parent_index = int((new_node_index - 1) // 2)
            
            while((parent_index >= 0 and (node_list[0] < self.queue[parent_index][0])) 
                  or (node_list[0] == self.queue[parent_index][0] 
                      and node_list[1] < self.queue[parent_index][1])): #Heapify, while parent is bigger than inserted node, swap them
                
                temp_parent_node = self.queue[parent_index]
                self.queue[parent_index] = node_list
                self.queue[new_node_index] = temp_parent_node
                new_node_index = parent_index
                parent_index = int((parent_index -1) // 2) #Will reach -1 if new node has lowest priority, adding and conditional
            
        return "Append Successful"

    def __contains__(self, key):
        """
        Containment Check operator for 'in'

        Args:
            key: The key to check for in the queue.

        Returns:
            True if key is found in queue, False otherwise.
        """

        return key in [n[-1] for n in self.queue]

    def __eq__(self, other):
        """
        Compare this Priority Queue with another Priority Queue.

        Args:
            other (PriorityQueue): Priority Queue to compare against.

        Returns:
            True if the two priority queues are equivalent.
        """

        return self.queue == other.queue

    def size(self):
        """
        Get the current size of the queue.

        Returns:
            Integer of number of items in queue.
        """

        return len(self.queue)

    def clear(self):
        """Reset queue to empty (no nodes)."""

        self.queue = []

    def top(self):
        """
        Get the top item in the queue.

        Returns:
            The first item stored in the queue.
        """

        return self.queue[0]
    
    def get_nodes(self):
        nodes = []
        for i in range(len(self.queue)):
            nodes.append(self.queue[i][2])
        return nodes

In [74]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestPriorityQueue

TestPriorityQueue("test_append_and_pop").test_append_and_pop(PriorityQueue)
TestPriorityQueue("test_fifo_property").test_fifo_property(PriorityQueue)

UnitTest passed successfully for "test_append_and_pop"!
UnitTest passed successfully for "test_fifo_property"!


### Breadth First Search

Breadth-First Search (BFS) is a foundational algorithm in computer science used to explore the nodes of a graph or tree structure systematically. It is particularly effective in scenarios where a solution or target node is expected to be closer to the root, as it explores all nodes at the present depth level before moving on to nodes at the next depth level.

#### Overview:

BFS operates with the underlying function:

$$f(n) = g(n)$$

Here, $f(n)$ represents the expansion priority of node $n$, and $g(n)$ quantifies the depth of node $n$ from the start node, counting the number of edges traversed. In BFS, all edges are considered to have equal weight, and nodes are explored based on their distance from the root node. This distance is effectively the shortest path in an unweighted graph when considering path cost purely in terms of the number of edges traversed.

#### BFS Algorithm Details:

1. **Initialization**: BFS begins at a specified root node and explores its neighbors first. It employs a queue data structure to manage the sequence of nodes explored, ensuring that nodes are expanded in the order they are discovered.
   
2. **Node Expansion**: Nodes are dequeued and their adjacent nodes are enqueued, if they have not been visited already. This step is repeated until the queue is empty or the goal is found. Each node is typically marked as visited to avoid revisiting and looping.

3. **Path Recovery**: To reconstruct the path from the start node to the goal, BFS can be modified to track the predecessor of each visited node. Once the goal node is found, the path can be easily reconstructed by following the predecessors from the goal node back to the start node.

4. **Complexity**: The time complexity of BFS is $O(V + E)$, where $V$ is the number of vertices and $E$ is the number of edges in the graph. The space complexity is also $O(V)$, as it needs to store all vertex states and the queue.

#### Completeness and Optimality

- **Completeness**: BFS is complete, meaning that it will find a solution if at least one exists. This property is crucial in scenarios where all possible states must be explored to find a solution, such as puzzles or finding any path in a maze.

- **Optimality**: When all edges have the same weight, BFS is also optimal. This means that the first time a goal node is reached, the path to it is the shortest possible in terms of the number of edges traversed. In scenarios where paths have different costs, BFS would not guarantee the least costly path unless all costs are equal.

#### Applications

BFS is widely used in situations requiring the exploration of all possible scenarios to a certain depth, pathfinding algorithms in simple maps, networking routines like broadcast protocols, and in the analysis of social networking graphs to find all entities within one connected component.


In [ ]:
def breadth_first_search(graph, start, goal):
    """
    Implement breadth-first-search.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """

    if start == goal: return []
    
    bfs_queue = [start]
    visited = set()
    parent_dir = {start: None}
    path = []
    
    while len(bfs_queue) > 0:
        current = bfs_queue.pop(0)

        if current == goal:
            #path = []
            while current is not None:
                path.insert(0, current)
                current = parent_dir[current]
            return path
        
        temp_list = []
        
        for neighbor in graph.neighbors(current):
            if neighbor not in visited and neighbor not in parent_dir:
                
                if neighbor == goal:
                    parent_dir[neighbor] = current
                    current = neighbor
                    while current is not None:
                        path.insert(0, current)
                        current = parent_dir[current]
                    return path
                    
                parent_dir[neighbor] = current
                temp_list.append(neighbor)
        
        temp_list = sorted(temp_list)
        
        for node in temp_list:
            bfs_queue.append(node)

    return None

In [ ]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestBFS

TestBFS("test_valid_paths").test_valid_paths(breadth_first_search)
TestBFS("test_optimal_paths").test_optimal_paths(breadth_first_search)
TestBFS("test_explored_counts").test_explored_counts(breadth_first_search)

### Uniform-cost search

Uniform Cost Search (UCS) is a more generalized form of Breadth-First Search that deals effectively with graphs where edges have varying costs. It prioritizes expanding the least costly paths first, making it particularly suitable for finding the shortest path in weighted graphs where costs between nodes vary.

#### Technical and Mathematical Details

UCS expands paths in order of increasing path cost. This strategy ensures that when a goal node is first reached, the path to it is guaranteed to be optimal with respect to the cost function used. This optimality and completeness are significant advantages of UCS, especially in complex and variable-cost environments.

#### Mathematical Overview:

The key function governing UCS is: $f(n) = g(n)$

where:
- $f(n)$ is the total cost of the path from the start node to node $n$,
- $g(n)$ is the cost from the start node to node $n$.

Unlike BFS, which assumes all path costs are equal, UCS extends this by allowing diverse and dynamic costs. It uses a priority queue (often implemented with a min-heap) to manage the expansion of nodes, ensuring that nodes with the lowest path cost are selected for expansion first.

#### Completeness and Optimality

- **Completeness**: UCS is complete, meaning it will always find a solution if one exists, as long as the cost of each step exceeds some small positive constant $ \epsilon $ (to prevent infinite loops).
  
- **Optimality**: The strategy is also optimal, guaranteeing that the first solution found is the least costly among all solutions. This optimality holds under the condition that the step costs are positive.

In [ ]:
def uniform_cost_search(graph, start, goal):
    """
    Uniform_cost_search.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    #Check if start == goal, and if return []
    if start == goal:
        return []
    
    #Assign priority queue and add start as first element (root) with priority of 0
    priority_q = PriorityQueue()
    start_tuple = (0, start)
    priority_q.append(start_tuple)
    visited = set()
    parent_dir = {start: ["stop", 0]}
    path = []
    
    #While top in our priority queue is not equal to the goal node
    while (priority_q.top()[2] != goal):
        
        #Assign current node that is being explored, pop it off the priority queue
        #Initialize temporary list = []
        current = priority_q.pop()
        temp_list = []
        
        #Loop through neighbors of current that are not in visited
        for neighbor in graph.neighbors(current[1]):
            
            #Save them to a temporary list of tuples with 
            #edge cost from current to neighbor + total edge cost to current
            
            if neighbor not in visited:
                
                total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parent_dir[current[1]][1]
                #Make current the parent of neighbor if not in parent_dir or smaller than existing entry
                if neighbor not in parent_dir:
                    
                    #Make current the parent of neighbor
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    temp_cost = total_edge_cost
                    temp_list.append((temp_cost, neighbor))
                    
                elif total_edge_cost < parent_dir[neighbor][1]:
                    
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    #edge_diff = parent_dir[neighbor][1] - total_edge_cost
                    priority_q.update(neighbor, total_edge_cost)
        
        #Mark current as visited 
        visited.add(current[1])
        
        sort_and_append(priority_q, temp_list)
        #Sort the list alphabetically based on their tuple[1] value which will be a single character type string
        #sorted_tuples = sorted(temp_list, key=lambda x: x[1])
        
        #Append the list values to the priority queue, which will handle the prioritization
        #for element in sorted_tuples:
            #priority_q.append(element)
            
        #If top of priority queue == goal
        #backtrack from goal node to start using parent_directory and build path, return path
        if priority_q.top()[2] == goal:
            back_track = goal
            
            while back_track != "stop":
                path.insert(0, back_track)
                back_track = parent_dir[back_track][0]
                
            return path
    
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

In [ ]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_ucs_romania", uniform_cost_search)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_ucs_romania", uniform_cost_search)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_ucs_romania", uniform_cost_search)

### A* search

A* Search is a powerful and widely used pathfinding and graph traversal algorithm that efficiently finds the shortest path from a start node to a goal node while minimizing the total cost (usually distance, time, or other resource). It extends Dijkstra's algorithm by using heuristics to estimate the cost from a node to the goal, which significantly speeds up the search process in large graphs by pruning paths that are unlikely to lead to the optimal solution.

#### Technical and Mathematical Details

A* search uses the following function to determine the order of node expansion:

$$f(n) = g(n) + h(n)$$

where:
- \(f(n)\) is the total estimated cost of the cheapest solution through node \(n\).
- \(g(n)\) is the cost from the start node to node \(n\), which is known.
- \(h(n)\) is a heuristic function estimating the lowest cost from node \(n\) to the goal.

#### Completeness and Optimality

- **Completeness**: A* is complete, meaning it will always find a solution if one exists, provided that the number of nodes in the graph is finite and the cost of moving between nodes is bounded by a positive constant.
  
- **Optimality**: A* is optimal if the heuristic function \(h(n)\) is admissible, meaning it never overestimates the actual cost to reach the nearest goal node from \(n\). Under these conditions, A* not only finds a solution, but it finds the optimal solution.


### Heuristic

For this implementation we'll be using the euclidean distance as heuristic to evaluate the distance from a node to the goal. The Euclidean distance metric is admissible when used as a heuristic in the A* search algorithm for problems where it represents the straight-line distance between the current node and the goal in a real-world layout, such as points on a map. This is because it never overestimates the actual shortest path cost between two points, adhering to the triangle inequality in metric spaces, thus ensuring that the heuristic is optimistic and guarantees the optimality of the A* search.

```python
def euclidean_dist_heuristic(graph, u, v):
    """
    Euclidean distance heuristic.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        u (str): Key for the first node to calculate from.
        v (str): Key for the second node to calculate to.

    Returns:
        Euclidean distance between the `u` node and the `v` node
        Round the result to 3 decimal places (if applicable)
    """

    x_diff = graph.nodes[u]['pos'][0] - graph.nodes[v]['pos'][0]
    y_diff = graph.nodes[u]['pos'][1] - graph.nodes[v]['pos'][1]

    return round(math.sqrt(x_diff**2 + y_diff**2), 3)
```

In [89]:
def euclidean_dist_heuristic(graph, u, v):
    """
    Euclidean distance heuristic.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        u (str): Key for the first node to calculate from.
        v (str): Key for the second node to calculate to.

    Returns:
        Euclidean distance between the `u` node and the `v` node
        Round the result to 3 decimal places (if applicable)
    """
    
    x_diff = graph.nodes[u]['pos'][0] - graph.nodes[v]['pos'][0]
    y_diff = graph.nodes[u]['pos'][1] - graph.nodes[v]['pos'][1]
    
    return round(math.sqrt(x_diff**2 + y_diff**2), 3)

In [90]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestEuclideanHeuristic

TestEuclideanHeuristic("test_euclidean_distance").test_euclidean_distance(euclidean_dist_heuristic)

UnitTest passed successfully for "test_euclidean_distance"!


#### A* Algorithm

In [93]:
def a_star(graph, start, goal, heuristic=euclidean_dist_heuristic):
    """
    Warm-up exercise: Implement A* algorithm.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    
    #Check if start == goal, and if return []
    if start == goal:
        return []
    
    #Assign priority queue and add start as first element (root) with priority of 0 + euclidean distance to goal (heuristic)
    priority_q = PriorityQueue()
    start_tuple = (heuristic(graph, start, goal), start)
    priority_q.append(start_tuple)
    visited = set()
    parent_dir = {start: ["stop", 0]}
    path = []
    
    #While top in our priority queue is not equal to the goal node
    while (priority_q.top()[2] != goal):
        
        #Assign current node that is being explored, pop it off the priority queue
        #Initialize temporary list = []
        current = priority_q.pop()
        temp_list = []
        
        #Loop through neighbors of current that are not in visited
        for neighbor in graph.neighbors(current[1]):
            
            #Save them to a temporary list of tuples with 
            #(edge cost from current to neighbor + euclidean distance from neighbor to goal (heuristic), node)
            
            if neighbor not in visited:
                
                total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parent_dir[current[1]][1]
                #Make current the parent of neighbor if not in parent_dir or smaller than existing entry
                if neighbor not in parent_dir:
                    
                    #Make current the parent of neighbor
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    temp_cost = total_edge_cost + heuristic(graph, neighbor, goal)
                    temp_list.append((temp_cost, neighbor))
                    
                elif total_edge_cost < parent_dir[neighbor][1]:
                    
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    #edge_diff = parent_dir[neighbor][1] - total_edge_cost
                    priority_q.update(neighbor, total_edge_cost + heuristic(graph, neighbor, goal))
        
        #Mark current as visited 
        visited.add(current[1])
        
        #Sort the list alphabetically based on their tuple[1] value which will be a single character type string
        sorted_tuples = sorted(temp_list, key=lambda x: x[1])
        
        #Append the list values to the priority queue, which will handle the prioritization
        for element in sorted_tuples:
            priority_q.append(element)
            
        #If top of priority queue == goal
        #backtrack from goal node to start using parent_directory and build path, return path
        if priority_q.top()[2] == goal:
            back_track = goal
            
            while back_track != "stop":
                path.insert(0, back_track)
                back_track = parent_dir[back_track][0]
                
            return path
        
    #raise NotImplementedError

In [94]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_a_star_null_romania", a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_a_star_null_romania", a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_a_star_null_romania", a_star, heuristic=null_heuristic)

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)

UnitTest passed successfully for "test_a_star_null_romania.test_valid_paths"!
UnitTest passed successfully for "test_a_star_null_romania.test_optimal_paths"!
UnitTest passed successfully for "test_a_star_null_romania.test_explored_counts"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_valid_paths"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_optimal_paths"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_explored_counts"!


### Bidirectional A* search 

Bi-directional A* conducts two synchronized A* searches, one from the start and another from the goal, which can significantly reduce the search space under certain conditions.

#### Stopping Condition:

This method uses two heuristics, $h_1(n)$ for the start-to-node heuristic and $h_2(n)$ for the goal-to-node heuristic. The searches meet effectively when:

$$g_1(n) + g_2(n) \geq f_{\text{min}}$$

where $g_1$ and $g_2$ are the costs from the start and goal to the node $n$ respectively, and $f_{\text{min}}$ is the smallest path cost found.

#### Ira Pohl's Contribution:

Pohl introduced a refined stopping condition for bidirectional search, focusing on ensuring that the combined paths' cost from start and goal to a meeting point $n$ does not exceed the best known path cost. This is mathematically captured as:

$$C(x) + C(y) - h(x, y) \leq C(p)$$

where $x$ and $y$ are nodes from the opposite ends of the search and $p$ is their meeting point.


In [338]:
def bidirectional_a_star(graph, start, goal,
                         heuristic=euclidean_dist_heuristic):
    """
    Bidirectional A*.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
        Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    #Start condition and initializations
    if start == goal: return []

    q_forward, q_backward = PriorityQueue(), PriorityQueue()
    visited_f, visited_b = set(), set()
    forward = True
    q_forward.append((0, start))
    q_backward.append((0, goal))
    parents_f = {start: ["stop", 0]}
    parents_b = {goal: ["stop", 0]}
    
    while (not visited_f.intersection(visited_b)): #while explored sets from forward and backward don't intersect
        
        if forward:   
            current = q_forward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_f:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_f[current[1]][1]
                    if neighbor not in parents_f: #If neighbor has no parent yet
                        parents_f[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_cost = total_edge_cost + heuristic(graph, neighbor, goal)
                        temp_list.append((temp_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_f[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_f[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_forward.update(neighbor, temp_cost) #Update queue
            
            visited_f.add(current[1]) #Add to forward set of visited
            sort_and_append(q_forward, temp_list) #Sort list alphabetically and append to forward queue
            
            forward = False
        
        #Backward 
        else: 
            current = q_backward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_b:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_b[current[1]][1]
                    if neighbor not in parents_b: #If neighbor has no parent yet
                        parents_b[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_cost = total_edge_cost + heuristic(graph, neighbor, start)
                        temp_list.append((temp_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_b[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_b[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_backward.update(neighbor, temp_cost) #Update queue
            
            visited_b.add(current[1]) #Add to forward set of visited
            sort_and_append(q_backward, temp_list) #Sort list alphabetically and append to forward queue
        
            forward = True
        
    #As while loop stopped, explored sets intersect on one element
    intersection_node = next(iter(visited_f.intersection(visited_b))) #Find intersection node
    
    crossover_points = list(visited_f.intersection(set((q_backward.get_nodes() + list(visited_b)))))

    path_costs = []
    for point in crossover_points:
        path_costs.append([point, parents_b[point][1] + parents_f[point][1]])
        
    path_costs = sorted(path_costs, key=lambda x: x[1])
    path_costs = [sublist for sublist in path_costs if intersection_node != sublist[0]]
    intersection_cost = parents_b[intersection_node][1] + parents_f[intersection_node][1]
    
    if intersection_cost > path_costs[0][1]:
        
        forward_path = backtrack_and_append(path_costs[0][0], parents_f)
        backward_path = backtrack_and_append(path_costs[0][0], parents_b)
    
        backward_path = backward_path[:-1] #Remove last element
        backward_path.reverse()
    
        path = forward_path + backward_path
        
        return path
    
    
    forward_path = backtrack_and_append(intersection_node, parents_f)
    backward_path = backtrack_and_append(intersection_node, parents_b)
    
    backward_path = backward_path[:-1] #Remove last element
    backward_path.reverse()
    
    path = forward_path + backward_path

    return path
   
    
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

def backtrack_and_append(intersection_node, parent_directory):
    path = []
    back_track = intersection_node
    while back_track != "stop":
        path.insert(0, back_track)
        back_track = parent_directory[back_track][0]
    return path

In [339]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)

UnitTest passed successfully for "test_bi_a_star_null_romania.test_valid_paths"!
UnitTest passed successfully for "test_bi_a_star_null_romania.test_optimal_paths"!
UnitTest passed successfully for "test_bi_a_star_null_romania.test_explored_counts"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_valid_paths"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_optimal_paths"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_explored_counts"!


### Tri-directional UCS search 

The Tri-Directional Search algorithm is an enhanced variant of pathfinding algorithms, designed to optimize node exploration by simultaneously initiating searches from three strategically chosen nodes—commonly the start, goal, and an intermediary node which acts as an additional source or target. This approach is predicated on minimizing the explored search space more efficiently than bi-directional search, potentially halving the exploration overhead under optimal conditions.

Tri-directional search leverages the theoretical framework of graph traversal to expedite convergence towards a solution by initiating multiple frontiers of exploration. Here, the graph is defined as G = (V, E) with V representing vertices and E representing edges. Each edge e connecting vertices u and v can optionally carry a weight w(u, v), which influences pathfinding in weighted scenarios.

#### Optimization Objective:

The core objective in the tri-directional approach is to minimize the total number of explored nodes, denoted as |N|, where N ⊆ V. This is achieved by optimizing the selection of the intermediary node and dynamically balancing the frontier expansion across the three nodes based on real-time heuristic evaluations.

#### Search Dynamics:

The search progresses through simultaneous expansion from three nodes using either BFS or DFS, potentially enhanced with heuristic functions akin to A*. For heuristic-driven searches, each node n in the frontier employs a cost function f(n) = g(n) + h(n), where:

- g(n) is the exact cost path from the start to n,
- h(n) is a heuristic estimate of the cost from n to the goal.

In the tri-directional setup, this formulation is adapted for three interacting frontiers, necessitating a triplet of heuristic functions each tailored for path estimation from their respective origins to their destinations.

#### Stopping Conditions

The algorithm's termination is predicated on the convergence of the search frontiers, which can be theoretically articulated through the lens of optimal stopping conditions.

##### Intersection-Based Stopping:

The primary stopping criterion is the detection of an intersection among the sets of nodes explored by each frontier, denoted as S1, S2, and S3. Mathematically, the search can cease when:

    S1 ∩ S2 ≠ ∅, or S1 ∩ S3 ≠ ∅, or S2 ∩ S3 ≠ ∅

This condition ensures that a connecting path exists, albeit not necessarily the shortest.

##### Cost-Based Stopping:

Extending on Ira Pohl’s insights from bidirectional search, the optimal path guarantee can be framed as follows: if P is a path connecting the start to the goal, then the search can terminate when the summed minimal heuristic evaluations of any two fronts, plus the interconnecting path cost, exceed the best-known path cost:

    min{f(x) | x ∈ Si} + min{f(y) | y ∈ Sj} + d(x, y) > C(P)

where i, j ∈ {1, 2, 3}, i ≠ j, and d(x, y) represents the direct path cost between nodes x and y.


In [464]:
def tridirectional_search(graph, goals):
    
    ### SET UP 
    if goals[0] == goals[1] == goals[2]: return [] 
    #test = False
    #if goals == ['b', 'm', 'o']:
        #test = True
    q1, q2, q3 = PriorityQueue(), PriorityQueue(), PriorityQueue()
    v1, v2, v3 = set(), set(), set()
    one, two, no_need_to_compare = True, True, False
    
    q1.append((0, goals[0]))
    q2.append((0, goals[1]))
    q3.append((0, goals[2]))

    parents_1 = {goals[0]: ["stop", 0]}
    parents_2 = {goals[1]: ["stop", 0]}
    parents_3 = {goals[2]: ["stop", 0]}
    
    ###############################
    
    ### INITIAL LOOP TO FIND FIRST INTERSECTION
    
    while ((q1.top()[2] not in v1.union(v2, v3)) and (q2.top()[2] not in v1.union(v2, v3)) and 
           (q3.top()[2] not in v1.union(v2, v3))):
        
        if one: #Search 1
            current = q1.pop()
            temp_list = []   
            
            
            if q1.size() != 0:
                if q1.top()[2] in v1.union(v2, v3):
                    v1.add(current[1]) #Add to forward set of visited
                    break
            
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v1:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_1[current[1]][1]
                    if neighbor not in parents_1: #If neighbor has no parent yet
                        parents_1[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_1[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_1[neighbor] = [current[1], total_edge_cost] #Update parent
                        q1.update(neighbor, total_edge_cost) #Update queue 
            v1.add(current[1]) #Add to forward set of visited
            sort_and_append(q1, temp_list) #Sort list alphabetically and append to queue
            one = False
        elif two: #Search 2 
            current = q2.pop()
            temp_list = []   
            
            if q2.size() != 0:
                if q2.top()[2] in v1.union(v2, v3):
                    v2.add(current[1]) #Add to forward set of visited
                    break
                    
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v2:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_2[current[1]][1]
                    if neighbor not in parents_2: #If neighbor has no parent yet
                        parents_2[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_2[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_2[neighbor] = [current[1], total_edge_cost] #Update parent
                        q2.update(neighbor, total_edge_cost) #Update queue
            v2.add(current[1]) #Add to forward set of visited
            sort_and_append(q2, temp_list) #Sort list alphabetically and append to queue
            two = False
        else: #Search 3 
            current = q3.pop()
            temp_list = []    
            
            if q3.size() != 0:
                if q3.top()[2] in v1.union(v2, v3):
                    v3.add(current[1]) #Add to forward set of visited
                    break
                    
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v3:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_3[current[1]][1]
                    if neighbor not in parents_3: #If neighbor has no parent yet
                        parents_3[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_3[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_3[neighbor] = [current[1], total_edge_cost] #Update parent
                        q3.update(neighbor, total_edge_cost) #Update queue
            v3.add(current[1]) #Add to forward set of visited
            sort_and_append(q3, temp_list) #Sort list alphabetically and append to queue
            one = True 
            two = True     
            
    ####################################
        
    ### OPTIMIZE FIRST PATH AB
    met = who_met_3(q1, q2, q3, v1, v2, v3, parents_1, parents_2, parents_3)
    eval('v' + str(met[1])).add(met[0])
    eval('v' + str(met[2])).add(met[0])
    ab_intersect = met[0]
    crossover_points = list(eval('v' + str(met[1])).intersection(set((eval('q' + str(met[2])).get_nodes() + 
                                                                             list(eval('v' + str(met[2])))))))
    path_costs = []
    for point in crossover_points:
        path_costs.append([point, eval('parents_' + str(met[2]))[point][1] + 
                           eval('parents_' + str(met[1]))[point][1]])
    path_costs = sorted(path_costs, key=lambda x: x[1])
    path_costs = [sublist for sublist in path_costs if ab_intersect != sublist[0]]
    intersection_cost = eval('parents_' + str(met[2]))[met[0]][1] + eval('parents_' + str(met[1]))[met[0]][1]
    if len(path_costs) != 0:
        if intersection_cost > path_costs[0][1]:
            ab_intersect = path_costs[0][0]
            eval('v' + str(met[1])).add(ab_intersect)
            eval('v' + str(met[2])).add(ab_intersect)
    ab_path_cost = eval('parents_' + str(met[2]))[ab_intersect][1] + eval('parents_' + str(met[1]))[ab_intersect][1]
    #####################################
    
    ### Continue Search with 3rd Search I
    temp1, temp2 = [met[1], met[2]], ['1', '2','3']
    result = [item for item in temp2 if item not in temp1] #result contains leftover search number
    q_continue = eval('q' + str(result[0]))
    parents_continue = eval('parents_' + str(result[0]))

    while (q_continue.top()[2] not in eval('v' + str(met[1])).union(eval('v' + str(met[2])))):
        current = q_continue.pop()
        temp_list = [] 
        """
        if q_continue.size() != 0:
            if q_continue.top()[2] in eval('v' + str(met[1])).union(eval('v' + str(met[2]))):
                sort_and_append(q_continue, temp_list)
                break
         """
        for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
            total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_continue[current[1]][1]
            if neighbor not in parents_continue: #If neighbor has no parent yet
                parents_continue[neighbor] = [current[1], total_edge_cost] #Assign parent
                temp_list.append((total_edge_cost, neighbor)) #Append to list    
            elif total_edge_cost < parents_continue[neighbor][1]: #If parent exists, but we found a cheaper path
                parents_continue[neighbor] = [current[1], total_edge_cost] #Update parent
                q_continue.update(neighbor, total_edge_cost) #Update queue               
        sort_and_append(q_continue, temp_list) #Sort list alphabetically and append to queue
        
    #####################################
    
    ### OPTIMIZE SECOND PATH C - 
    c_search_x_initial = who_met_2(q_continue.top()[2], eval('v' + str(met[1])), met[1], eval('v' + str(met[2])), met[2])  
    crossover_points = list(eval('v' + str(c_search_x_initial)).intersection(set((q_continue.get_nodes()))))
    path_costs = []
    for point in crossover_points:
        path_costs.append([point, parents_continue[point][1] + eval('parents_' + str(c_search_x_initial))[point][1]])    
    path_costs = sorted(path_costs, key=lambda x: x[1])
    path_costs = [sublist for sublist in path_costs if q_continue.top()[2] != sublist[0]]
    intersection_cost = parents_continue[q_continue.top()[2]][1] + eval('parents_' + str(c_search_x_initial))[q_continue.top()[2]][1]
    intersect_node_c_x_initial = q_continue.top()[2]
    if len(path_costs) != 0:
        if intersection_cost > path_costs[0][1]:
            intersect_node_c_x_initial = path_costs[0][0] #Update intersect node 
    c_x_path_cost_1 = parents_continue[intersect_node_c_x_initial][1] + eval('parents_' + str(c_search_x_initial))[intersect_node_c_x_initial][1]
    
    ######################################

    ### Continue Search with 3rd Search II
    options = [met[1], met[2]]
    c_search_x_second = [item for item in options if item != c_search_x_initial][0]
    
    if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
        no_need_to_compare = True
        
    while (q_continue.top()[2] not in eval('v' + str(c_search_x_second))) and not no_need_to_compare:
        if q_continue.size() == 1:
            if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
                break
                
        current = q_continue.pop()
        temp_list = []      
        
        if q_continue.size() != 0:
            if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
                break
                
        for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
            total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_continue[current[1]][1]
            if neighbor not in parents_continue: #If neighbor has no parent yet
                parents_continue[neighbor] = [current[1], total_edge_cost] #Assign parent
                temp_list.append((total_edge_cost, neighbor)) #Append to list    
            elif total_edge_cost < parents_continue[neighbor][1]: #If parent exists, but we found a cheaper path
                parents_continue[neighbor] = [current[1], total_edge_cost] #Update parent
                q_continue.update(neighbor, total_edge_cost) #Update queue                 
        sort_and_append(q_continue, temp_list) #Sort list alphabetically and append to queue    
        
    if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
        no_need_to_compare = True
    
    #######################################
    
    ### OPTIMIZE THIRD PATH C - (if applicable)
    if not no_need_to_compare:
        crossover_points = list(eval('v' + str(c_search_x_second)).intersection(set((q_continue.get_nodes()))))
        path_costs = []
        intersect_node_c_x_second = q_continue.top()[2]
        for point in crossover_points:
            path_costs.append([point, parents_continue[point][1] + eval('parents_' + str(c_search_x_second))[point][1]])    
        path_costs = sorted(path_costs, key=lambda x: x[1])
        path_costs = [sublist for sublist in path_costs if intersect_node_c_x_second != sublist[0]]
        if not no_need_to_compare:
            intersection_cost = parents_continue[intersect_node_c_x_second][1] + eval('parents_' + str(c_search_x_second))[intersect_node_c_x_second][1]
            if len(path_costs) != 0:
                if intersection_cost > path_costs[0][1]:
                    intersect_node_c_x_second = path_costs[0][0] #Update intersect node 
            c_x_path_cost_2 = parents_continue[intersect_node_c_x_second][1] + eval('parents_' + str(c_search_x_second))[intersect_node_c_x_second][1]
    
    ######################################
    
    ### Build Path
    #Case 1: CB was determined to be longer than max of (AB, AC) and therefore never searched for
    if no_need_to_compare:
        c_path = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp_path = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp_path.reverse()
        c_path = c_path + temp_path
        if c_search_x_initial == met[1]:
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path.reverse()
            ab_path = ab_path + b_path
        else: 
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path.reverse()
            ab_path = ab_path + b_path
            
        full_path = c_path + ab_path
        result_path = filter_nodes(full_path)
        return result_path
    
    #Case 2: CB was found. If CA <= CB
    if c_x_path_cost_1 <= c_x_path_cost_2:
        c_path = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp_path = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp_path.reverse()
        c_path_1 = c_path + temp_path
        if (c_x_path_cost_2 < ab_path_cost) and (c_x_path_cost_1 != c_x_path_cost_2):
            temp1 = backtrack_and_append(intersect_node_c_x_second, eval('parents_' + str(c_search_x_second)))
            temp2 = backtrack_and_append(intersect_node_c_x_second, parents_continue)
            temp2.reverse()
            c_path_2 = temp1 + temp2
            full_path = c_path_2 + c_path_1
            result_path = filter_nodes(full_path)
            return result_path
        else:
            if c_search_x_initial == met[1]:
                ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
                b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
                b_path.reverse()
                ab_path = ab_path + b_path
            else: 
                ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
                b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
                b_path.reverse()
                ab_path = ab_path + b_path
            full_path = c_path_1 + ab_path
            result_path = filter_nodes(full_path)
            return result_path
    
    #Case 3: CB was found. If CA > CB
    c_path = backtrack_and_append(intersect_node_c_x_second, parents_continue)
    temp_path = backtrack_and_append(intersect_node_c_x_second, eval('parents_' + str(c_search_x_second)))
    temp_path.reverse()
    c_path = c_path + temp_path
    if (c_x_path_cost_1 < ab_path_cost) and (c_x_path_cost_1 != c_x_path_cost_2):
        temp1 = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp2 = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp2.reverse()
        c_path_2 = temp1 + temp2
        full_path = c_path_2 + c_path
        result_path = filter_nodes(full_path)
        return result_path
    else:
        if c_search_x_second == met[1]:
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path.reverse()
            ab_path = ab_path + b_path
        else: 
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path.reverse()
            ab_path = ab_path + b_path
        
        full_path = c_path + ab_path
        result_path = filter_nodes(full_path)
        return result_path
        
##########################################  
##########################################  
##########################################  

def filter_nodes(full_path):
    result = []
    for i in range(len(full_path) - 1):
        if full_path[i] != full_path[i + 1]:
            result.append(full_path[i])
    result.append(full_path[-1])
    return result
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

def backtrack_and_append(intersection_node, parent_directory):
    path, counter = [], 0
    back_track = intersection_node
    while back_track != "stop":
        path.insert(0, back_track)
        back_track = parent_directory[back_track][0]
    return path
    
def who_met_2(q1, v2,num2, v3,num3):
    if q1 in v2:
        return str(num2)
    elif q1 in v3:
        return str(num3)
    return None

def who_met_3(q1, q2, q3, v1, v2, v3, p1, p2, p3):
    intersections = []
    if q1.top()[2] in v2:
        intersections.append((q1.top()[2], '1', '2', p1[q1.top()[2]][1] + p2[q1.top()[2]][1]))
    if q1.top()[2] in v3:
        intersections.append((q1.top()[2], '1', '3', p1[q1.top()[2]][1] + p3[q1.top()[2]][1]))
    if q2.top()[2] in v1:
        intersections.append((q2.top()[2], '2', '1', p2[q2.top()[2]][1] + p1[q2.top()[2]][1]))
    if q2.top()[2] in v3:
        intersections.append((q2.top()[2], '2', '3', p2[q2.top()[2]][1] + p3[q2.top()[2]][1]))
    if q3.top()[2] in v1:
        intersections.append((q3.top()[2], '3', '1', p3[q3.top()[2]][1] + p1[q3.top()[2]][1]))
    if q3.top()[2] in v2:
        intersections.append((q3.top()[2], '3', '2', p3[q3.top()[2]][1] + p2[q3.top()[2]][1]))
    if intersections:
        return min(intersections, key=lambda x: x[3])
    else:
        return None

### Conclusion

The implementation of various search algorithms, particularly focusing on bi-directional and tri-directional searches, marks a significant exploration into the complexities of artificial intelligence problem-solving techniques. While fundamental algorithms like Breadth-First Search, A*, and Dijkstra's provide a strong foundation, extending these concepts into bi-directional and tri-directional contexts introduces a layer of complexity that challenges both the understanding and the skills of any algorithm designer.

#### Key Achievements and Challenges

The primary challenge in extending traditional search algorithms to bi-directional and tri-directional search is not merely about getting them to work but optimizing them to minimize the count of explored nodes and improve on time and space complexities. This project highlights the intricacies involved in designing such algorithms to efficiently converge upon solutions by meeting at a mutual node from multiple directions, thereby significantly reducing the search space and potentially the computational load.

#### Optimization and Efficiency

The development and refinement of bi-directional and tri-directional search algorithms have underscored the importance of efficient algorithm design in AI. These methods can vastly outperform single-direction search algorithms in complex and large-scale problem spaces by strategically exploring from multiple points simultaneously, thus approaching the solution more rapidly.

#### Reflections and Future Directions

This exercise has not only reinforced the value of foundational search algorithms but also demonstrated the advanced capabilities required to implement more sophisticated techniques effectively. For future work, there remains ample scope to refine these algorithms further, enhancing their efficiency and adaptability to a broader range of problems. Investigating the integration of more dynamic heuristic functions and exploring machine learning approaches to predict optimal convergence points in tri-directional searches could provide valuable directions for research.

In conclusion, this project has been an enlightening journey into the depths of algorithmic problem-solving within AI. The successful implementation of bi-directional and tri-directional searches, in particular, offers a robust framework for tackling complex search problems, paving the way for more innovative solutions in the field of artificial intelligence.